# 注意：本文件中所有nn使用torch实现（个人对torch更熟悉）
## 准备数据

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 准备数据
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

## 建立模型

In [4]:
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 10)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return self.log_softmax(x)

model = FNN().to(device)
criterion = nn.NLLLoss()  # 负对数似然损失，配合LogSoftmax
optimizer = optim.SGD(model.parameters(), lr=1e-5)

# 写训练和测试函数

In [5]:
# 训练函数
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 测试函数
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

## 实际训练

In [6]:
for epoch in range(1, 51):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.375385
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.352809
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.391284
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.426278
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.356529
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.364251
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.339247
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.367275
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.331182
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.377909

Test set: Average loss: 0.0024, Accuracy: 966/10000 (10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.342038
Train Epoch: 2 [6400/60000 (11%)]	Loss: 2.315703
Train Epoch: 2 [12800/60000 (21%)]	Loss: 2.341734
Train Epoch: 2 [19200/60000 (32%)]	Loss: 2.363348
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.399442
Train Epoch: 2 [32000/60000 (53%)]	Loss: 2.404377
Train Epoch: 2 [38400/60000 (64%)]	Loss: 2.359919
Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.309893
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.322016
Tr

Train Epoch: 15 [51200/60000 (85%)]	Loss: 2.106243
Train Epoch: 15 [57600/60000 (96%)]	Loss: 2.125760

Test set: Average loss: 0.0021, Accuracy: 3081/10000 (31%)

Train Epoch: 16 [0/60000 (0%)]	Loss: 2.143304
Train Epoch: 16 [6400/60000 (11%)]	Loss: 2.158323
Train Epoch: 16 [12800/60000 (21%)]	Loss: 2.142863
Train Epoch: 16 [19200/60000 (32%)]	Loss: 2.165056
Train Epoch: 16 [25600/60000 (43%)]	Loss: 2.123098
Train Epoch: 16 [32000/60000 (53%)]	Loss: 2.139466
Train Epoch: 16 [38400/60000 (64%)]	Loss: 2.143799
Train Epoch: 16 [44800/60000 (75%)]	Loss: 2.072395
Train Epoch: 16 [51200/60000 (85%)]	Loss: 2.106961
Train Epoch: 16 [57600/60000 (96%)]	Loss: 2.109974

Test set: Average loss: 0.0021, Accuracy: 3285/10000 (33%)

Train Epoch: 17 [0/60000 (0%)]	Loss: 2.091527
Train Epoch: 17 [6400/60000 (11%)]	Loss: 2.179228
Train Epoch: 17 [12800/60000 (21%)]	Loss: 2.142626
Train Epoch: 17 [19200/60000 (32%)]	Loss: 2.106802
Train Epoch: 17 [25600/60000 (43%)]	Loss: 2.117981
Train Epoch: 17 [32000/

Train Epoch: 30 [19200/60000 (32%)]	Loss: 1.967823
Train Epoch: 30 [25600/60000 (43%)]	Loss: 1.943847
Train Epoch: 30 [32000/60000 (53%)]	Loss: 1.918602
Train Epoch: 30 [38400/60000 (64%)]	Loss: 1.916362
Train Epoch: 30 [44800/60000 (75%)]	Loss: 1.916871
Train Epoch: 30 [51200/60000 (85%)]	Loss: 1.915438
Train Epoch: 30 [57600/60000 (96%)]	Loss: 1.933088

Test set: Average loss: 0.0019, Accuracy: 5157/10000 (52%)

Train Epoch: 31 [0/60000 (0%)]	Loss: 1.952710
Train Epoch: 31 [6400/60000 (11%)]	Loss: 1.881994
Train Epoch: 31 [12800/60000 (21%)]	Loss: 1.926744
Train Epoch: 31 [19200/60000 (32%)]	Loss: 1.846857
Train Epoch: 31 [25600/60000 (43%)]	Loss: 1.831645
Train Epoch: 31 [32000/60000 (53%)]	Loss: 1.953084
Train Epoch: 31 [38400/60000 (64%)]	Loss: 1.919778
Train Epoch: 31 [44800/60000 (75%)]	Loss: 1.910091
Train Epoch: 31 [51200/60000 (85%)]	Loss: 1.947654
Train Epoch: 31 [57600/60000 (96%)]	Loss: 1.913446

Test set: Average loss: 0.0019, Accuracy: 5252/10000 (53%)

Train Epoch: 32 [

Train Epoch: 44 [57600/60000 (96%)]	Loss: 1.764426

Test set: Average loss: 0.0017, Accuracy: 6107/10000 (61%)

Train Epoch: 45 [0/60000 (0%)]	Loss: 1.792081
Train Epoch: 45 [6400/60000 (11%)]	Loss: 1.753014
Train Epoch: 45 [12800/60000 (21%)]	Loss: 1.667024
Train Epoch: 45 [19200/60000 (32%)]	Loss: 1.746590
Train Epoch: 45 [25600/60000 (43%)]	Loss: 1.713296
Train Epoch: 45 [32000/60000 (53%)]	Loss: 1.757093
Train Epoch: 45 [38400/60000 (64%)]	Loss: 1.746684
Train Epoch: 45 [44800/60000 (75%)]	Loss: 1.814887
Train Epoch: 45 [51200/60000 (85%)]	Loss: 1.688820
Train Epoch: 45 [57600/60000 (96%)]	Loss: 1.756308

Test set: Average loss: 0.0017, Accuracy: 6156/10000 (62%)

Train Epoch: 46 [0/60000 (0%)]	Loss: 1.707843
Train Epoch: 46 [6400/60000 (11%)]	Loss: 1.656018
Train Epoch: 46 [12800/60000 (21%)]	Loss: 1.694059
Train Epoch: 46 [19200/60000 (32%)]	Loss: 1.726766
Train Epoch: 46 [25600/60000 (43%)]	Loss: 1.670163
Train Epoch: 46 [32000/60000 (53%)]	Loss: 1.750361
Train Epoch: 46 [38400/